# Using EcoFOCIpy to process raw field data

## ____ Template for cruise (pre 2020)

## BTL Data + Nutrient Data

This is a streamlined version of generation routines to merge bottle data and Mordy Nut. Lab Nutrient Data for long term archive

<div class="warning" style='background-color:#E9D8FD; color: #69337A; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>A template for Nutrient Lab ASCII files to NETCDF</b></p>
<p style='margin-left:1em;'>
Populate the necessary paths in the following cells.</p>
<p style='margin-bottom:1em; margin-right:1em; text-align:right; font-family:Georgia'> <b>- Shaun Bell</b>
</p></span>
</div>


In [2]:
import yaml
import glob
import pandas as pd
import os
import xarray as xa

import ecofocipy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import ecofocipy.io.ncCFsave as ncCFsave
import ecofocipy.metaconfig.load_config as load_config

In [3]:
sample_data_dir = '/Users/bell/ecoraid/2007/CTDcasts/tn211/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [4]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
datafile = '/Users/bell/ecoraid/2007/CTDcasts/tn211/final_data/btl/' #<- point to cruise and process all files within
nutdatafile = sample_data_dir+'working/DiscreteNutrients/TN211 Nutrient Data.txt' #<- point to cruise and process all files within
cruise_name = 'tn211' #no hyphens
cruise_meta_file = sample_data_dir+'logs/TN211.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDpNutsWOCE.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################
#init and load data
filename_list = sorted(glob.glob(datafile + '*.nc'))

In [5]:
cruise_data = {}

for i in filename_list:
    cast = 'ctd'+i.split('.')[0].split('c')[-1].replace('_','.')
    cruise_data.update({cast:xa.load_dataset(datafile + i.split('/')[-1],decode_times=False)})

In [8]:
cruise_data['ctd001.btl'].to_dataframe().reset_index()

KeyError: 'ctd001.btl'

## Load csv Nutrient File

In [33]:
nut_data = pd.read_csv(nutdatafile,delimiter='\t')
nut_data

,Cast,Niskin,PO4 (uM),PO4 Flag,Sil (uM),Sil Flag,NO3 (uM),NO3 Flag,NO2 (uM),NO2 Flag,NH4 (uM),NH4 Flag
0,2,8,0.720,2,10.7,2,6.5,2,0.17,2,0.29,2
1,2,7,0.719,2,10.9,2,6.7,2,0.17,2,0.47,2
2,2,6,0.790,2,12.7,2,7.7,2,0.18,2,0.49,2
3,2,5,0.839,2,14.1,2,8.4,2,0.19,2,0.74,2
4,2,4,0.798,2,14.4,2,8.2,2,0.20,2,0.29,2
...,...,...,...,...,...,...,...,...,...,...,...,...
99,24,4,1.665,2,34.4,2,15.0,2,0.10,2,2.25,2
100,24,5,0.937,2,17.2,2,1.7,2,0.02,2,0.52,2
101,24,6,0.673,2,10.7,2,1.2,2,0.01,2,0.56,2
102,24,7,0.325,2,2.2,2,0.0,2,0.00,2,0.01,2


## Merge Bottle and Nutrient Data but drop non nutrient vars?

<div class="warning" style='background-color:#ffcccb; color: #FF0000; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>WARNING</b></p>
<p style='margin-left:1em;'>bottle/niskin and rosette position should be the same but can be different (example, bottles are labeled sequentially but a rosette position is skipped due to balancing or other instruments.</p>
</div>

In [34]:
keep_param = ['BTL_103','dep'] #sometimes prsm

for cast,cdata in cruise_data.items():
    try:
        matchcast = int((cast.split('.')[0]).lower().split('ctd')[-1])
        cruise_data[cast] = pd.merge(nut_data[nut_data['Cast']==matchcast],cdata.to_dataframe().reset_index()[keep_param],right_on='BTL_103',left_on='Niskin').set_index('BTL_103').drop(columns=['Cast'])
    except:
        print('something is wrong')

## Add Deployment meta information

In [35]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [36]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)

## Add institutional meta-information


In [37]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [38]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            'Sil (uM)':'SI',
                            'PO4 (uM)':'PO4',
                            'NO2 (uM)':'NO2', 
                            'NO3 (uM)':'NO3',
                            'NH4 (uM)':'NH4',
                            'BTL_103':'BTLID',
                            # 'prdm':'pressure',
                            'dep':'depth',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data[cast].index = cruise_data[cast].index.rename('bottle')
        
        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.lower().split('d')[-1].split('.')[0].zfill(3)+'_nut.nc',format="NETCDF3_CLASSIC")
    except KeyError:
        print(f'Skipping {cast}')
    except RuntimeError:
        print(f'Skipping & Removing {cast}')
        os.remove(path=cruise_name+'c'+cast.lower().split('d')[-1].split('.')[0].zfill(3)+'_nut.nc')

Skipping & Removing ctd012.btl
Skipping & Removing ctd013.btl
Skipping & Removing ctd014.btl
Skipping & Removing ctd015.btl
Skipping & Removing ctd017.btl
Skipping & Removing ctd018.btl
Skipping & Removing ctd019.btl
Skipping & Removing ctd020.btl
Skipping & Removing ctd025.btl


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history

In [39]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:    (bottle: 0, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle     (bottle) float64 
  * latitude   (latitude) float64 55.73
  * longitude  (longitude) float64 -170.2
  * time       (time) datetime64[ns] 2012-06-01T18:53:00
Data variables:
    Niskin     (latitude, longitude, time, bottle) int64 
    PO4        (latitude, longitude, time, bottle) float64 
    PO4 Flag   (latitude, longitude, time, bottle) int64 
    SI         (latitude, longitude, time, bottle) float64 
    Sil Flag   (latitude, longitude, time, bottle) int64 
    NO3        (latitude, longitude, time, bottle) float64 
    NO3 Flag   (latitude, longitude, time, bottle) int64 
    NO2        (latitude, longitude, time, bottle) float64 
    NO2 Flag   (latitude, longitude, time, bottle) int64 
    NH4        (latitude, longitude, time, bottle) float64 
    NH4 Flag   (latitude, longitude, time, bottle) int64 
    depth      (latitude, longitude, time, bottle) float64 
Attributes: (12/44)
    CruiseID:                      DY1205
    CruiseID_Historic:             2DY12
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        R/V Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2022-12-27T16:31:59Z
    date_modified:                 
    QC_indicator:                  excellent